# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [2]:
import pandas as pd
import numpy as np
from scipy import stats

In [3]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [7]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [8]:
# number of callbacks for white-sounding names
sum(data[data.race=='w'].call)

235.0

In [5]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4870 entries, 0 to 4869
Data columns (total 65 columns):
id                    4870 non-null object
ad                    4870 non-null object
education             4870 non-null int8
ofjobs                4870 non-null int8
yearsexp              4870 non-null int8
honors                4870 non-null int8
volunteer             4870 non-null int8
military              4870 non-null int8
empholes              4870 non-null int8
occupspecific         4870 non-null int16
occupbroad            4870 non-null int8
workinschool          4870 non-null int8
email                 4870 non-null int8
computerskills        4870 non-null int8
specialskills         4870 non-null int8
firstname             4870 non-null object
sex                   4870 non-null object
race                  4870 non-null object
h                     4870 non-null float32
l                     4870 non-null float32
call                  4870 non-null float32
city        

## Answers:
Q1: T-test is approprate to be used. Although we have a large sample here, but the variance of the population is still unknown. CLT applys, since the sample's size is large and all observations are independent.

Q2: Null hypothesis is ' The means of black and white applicants is the same'.

In [22]:
# Call back rates
w = sum(data[data.race=='w'].call)
b = sum(data[data.race=='b'].call)
n_w = len(data[data.race=='w'])
n_b = len(data[data.race=='b'])

p_b = b / n_b
p_w = w / n_w

print(p_b,p_w)

0.06447638603696099 0.09650924024640657


In [23]:
# Compute the Standard error.
s_e = np.sqrt((p_b*(1-p_b)/n_b)+(p_w*(1-p_w)/n_w))
s_e

0.0077833705866767544

In [25]:
# Compute 95% confidence interval
crit = stats.norm.isf([0.025,0.005])
c_i = [p_b-p_w-crit[0]*s_e, p_b-p_w+crit[0]*s_e]
c_i

[-0.04728798023766042, -0.01677772818123075]

In [28]:
# conduct two-sample t-test
import math

p_h = (n_b*p_b+n_w*p_w)/(n_b+n_w) # joint callback
std_h = math.sqrt(p_h*(1-p_h)*((1/n_b)+(1/n_w))) # joint standard deviation

test_stat = (p_b-p_w)/std_h
p_val = stats.t.sf(abs(test_stat), df=len(data)-1)*2


print(test_stat,p_val)

-4.108412152434346 4.0493178875903686e-05


In [30]:
def draw_bs_reps(data, func, size=1):
    """Draw bootstrap replicates."""

    # Initialize array of replicates: bs_replicates
    bs_replicates = np.empty(size)

    # Generate replicates
    for i in range(size):
        bs_replicates[i] = func(np.random.choice(data, size=len(data)))

    return bs_replicates


In [37]:
b1 = data.call[(data.race == 'b')]
w1 = data.call[(data.race == 'w')]

In [39]:
# Difference of means
diff_means = np.mean(b1) - np.mean(w1)
diff_means

-0.03203285485506058

In [40]:
# Generate shiftted arrays
m_all = np.mean(data.call)

b_s = b1 - np.mean(b1) + m_all
w_s = w1 - np.mean(w1) + m_all

In [41]:
# Conduct 10,000 bootstrap replicates
bs_replicates_b = draw_bs_reps(b_s, np.mean, 10000)
bs_replicates_w = draw_bs_reps(w_s, np.mean, 10000)

In [44]:
# Difference of replicates' means
bs_replicates_d = bs_replicates_b - bs_replicates_w
bs_replicates_d

array([-2.46404111e-03,  2.98023224e-08,  2.46409327e-03, ...,
       -5.74945658e-03, -2.05335766e-03,  1.02669671e-02])

In [47]:
# Compute P-value
p_val = np.sum(bs_replicates_d <= diff_means) / len(bs_replicates)
print('P-value:', p_val)

P-value: 0.0


## Answers
### Q4: According to the resualts, there is statistical significance between the callback rates among black-sounding and white-sounding. Therefore, the racial discrimination does exisit.

### Q5: No, the conclusion comes from the given data, doesn't imply that the race is the most important factor in callback. In order to get higher accurance, we need consider more variables, such as education, exprience, maybe even the layout of their resume.